# 既存のデータセットの活用

---
## 目的
PyTorch（`torchvision`）における既存のデータセットクラスとその使用方法について理解する．

## 準備

### Google Colaboratoryの設定確認・変更
本チュートリアルではPyTorchを利用してニューラルネットワークの実装を確認，学習および評価を行います．
**GPUを用いて処理を行うために，上部のメニューバーの「ランタイム」→「ランタイムのタイプを変更」からハードウェアアクセラレータをGPUにしてください．**

## モジュールのインポート
はじめに，必要なモジュールをインポートしたのち，GPUを使用した計算が可能かどうかを確認します．

### GPUの確認
GPUを使用した計算が可能かどうかを確認します．

`GPU availability: True`と表示されれば，GPUを使用した計算をPyTorchで行うことが可能です．
Falseとなっている場合は，上記の「Google Colaboratoryの設定確認・変更」に記載している手順にしたがって，設定を変更した後に，モジュールのインポートから始めてください．



In [ ]:
# モジュールのインポート
import os
import numpy as np
import torch
import torch.nn as nn

import torchvision
import torchvision.transforms as transforms

# GPUの確認
use_cuda = torch.cuda.is_available()
print('Use CUDA:', use_cuda)

## 既存のデータセット

PyTorchのコンピュータビジョン用ライブラリであるtorchvisionには，様々な既存のデータセットを簡単に利用できるよう，データセットクラスが定義されています．
以下では，画像分類タスクの代表的なデータセットである

* MNIST
* CIFAR10
* CIFAR100
* ImageNet

データセットについて紹介します．

その他のデータセットクラスについては，[torchvision.datasetsのリファレンスページ](https://pytorch.org/vision/stable/datasets.html)をご確認ください．

---
## MNISTデータセット

[MNISTデータセット](http://yann.lecun.com/exdb/mnist/)は，0~9までの手書き数字を分類するためのデータセットであり，Deep Learningを学ぶ際のHello Worldのような位置づけのデータセットです．
これまでにほかのノートブックを使用したことのある方であれば，ご存知だと思いますので，詳細については割愛します．

### 使い方
MNISTデータセットは`torchvision.datasets.MNIST`クラスを呼び出すことで，データのダウンロードから読み込みまでを自動的に行い，すぐに使用することが可能です．

`root`はMNISTデータセットのあるディレクトリまでのパス，`train`は学習用データを使用するかどうか（`False`の場合はテストデータを使用），`download`は`root`にデータセットがない場合に自動的にダウンロードするかどうかを示す引数です．


`transform`は画像データに対して行う前処理を定義して受け渡す引数です．
これ以降で紹介するデータセットについても`transform`は引数として存在しており，各データに対する前処理を簡単に定義することができます．
`transform`の定義としては，`torchvision.transforms`を使用します．
`torchvision.transforms`には様々な前処理のためのクラスが用意されています．
代表的なものとしては，

* `ToTensor()`: 呼び出した画像データ（Numpy arrayまたはPILフォーマットの画像データ）を`torch.Tensor`（PyTorchの配列型式）に変換するクラス
* `Normalize()`: 画像データを正規化するクラス
* `Grayscale()`: 画像をグレースケール変換するクラス
* `RandomCrop()`: 画像をランダムクロップするクラス（Data Augmentation）

などです．

今回は，読み込んだデータを`Tensor`型の配列に変換する`ToTensor()`を前処理として使用します．

そのほかにも様々な前処理がありますので，興味のある方は[torchvision.transformsのリファレンスページ](https://pytorch.org/vision/stable/transforms.html)より確認してください．

また，`torchvision.transforms`のより詳細な使い方に関しては，[データ拡張（Data Augmentation）](02_dnn_simple_pytorch/augmentation.ipynb)にて説明を行っていますので，そちらを参照してください． [![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/machine-perception-robotics-group/MPRGDeepLearningLectureNotebook/blob/master/02_dnn_simple_pytorch/augmentation.ipynb)

In [ ]:
train_data = torchvision.datasets.MNIST(root="./", train=True, transform=transforms.ToTensor(), download=True)
test_data = torchvision.datasets.MNIST(root="./", train=False, transform=transforms.ToTensor(), download=True)

print(type(train_data.data), type(train_data.targets))
print(type(test_data.data), type(test_data.targets))
print(train_data.data.size(), train_data.targets.size())
print(test_data.data.size(), test_data.targets.size())

### データセットクラスの情報とデータの呼び出し

`MNIST()`などのtorchvisionのデータセットクラスは，呼び出し後，クラスインスタンスのインデックスを指定することで，画像データとラベルを呼び出すことができます．

**データセットのサンプル数**

`len()`を用いることで，データセット内のサンプル数を確認することができます．


**インデックスを用いたデータの呼び出し**

`train_data[0]`のようにインデックスを指定することで，対応する番号の画像データとそのラベルを呼び出すことが可能です．
インデックスの上限は`len()`で表示した，サンプル数に対応しています．

呼び出した画像データの配列の形状を`size()`を用いて確認してみます．
その場合．配列のサイズが`[1, 28, 28]`となっていることがわかります．
これは，前処理の`ToTensor()`を使用するようにしているためであり，`ToTensor()`を用いることで，`Tensor`型の配列に変換すると同時に配列の型式を`[チャンネル, 縦, 横]`の順番に入れ替える配列操作も同時に行っているためです．

In [ ]:
# データ数の確認
print("The number of training data:", len(train_data))
print("The number of test data:", len(test_data))

# インデックスを用いたデータの呼び出し
image_data, label_data = train_data[0]
print(type(image_data), image_data.size())
print(type(label_data), label_data)

### DataLoaderの定義

上記のデータセットクラスをそのまま活用することで，ネットワークの学習や評価を行うことも可能ですが，
データの呼び出しを便利にするために，`torch.utils.data.DataLoader`を活用する方法を紹介します．

`DataLoader`には次のような引数があります．

* `dataset`に使用したいデータセットクラスのインスタンスを指定します．
* `batch_size`で，一度の呼び出したいサンプル数（ミニバッチサイズ）を指定します．
* `shuffle`で，ランダムにサンプルを抽出するかどうかを選択します．

そのほかにも様々な引数がありますが，興味のある方は[DataLoaderのリファレンスページ](https://pytorch.org/docs/stable/data.html)をご確認ください．

以下のプログラムでは，MNISTデータセットをDataLoaderへと受け渡して，実際にデータを呼び出してみます．
定義したDataLoaderは，データセットのサンプル数と指定した`batch_size`から，全てのデータが一通り呼び出される回数を自動的に計算します．
そして，for文に用いることで，データとラベルをループで呼び出すことが可能です．

呼び出されたデータを確認すると，画像データの配列サイズは，`[10, 1, 28, 28]`となっていることがわかります．
これは，`[ミニバッチサイズ，チャンネル，縦，横]`という配列サイズになっています．
DataLoaderでは，1次元目がバッチサイズとなり，その後ろはDatasetクラスでインデックスを用いたデータの呼び出しの際の配列の形状が維持されます．

以前のノートブックから，PyTorchでのネットワーク（畳み込み層）へのデータの入力の形式は`[ミニバッチサイズ，チャンネル，縦，横]`でした．
そのため，データセットクラスの`transform`に`ToTensor()`を指定して，DataLoaderへ受け渡して使用すると．自動的にネットワークへ入力することのできる配列になった状態でデータを受け取ることが可能です．

In [ ]:
# DataLoaderの定義
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=10, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=10, shuffle=False)

for image, label in train_loader:
    print(image.size())
    print(label.size(), label)
    break

---
## CIFAR10データセット

[CIFAR10/100データセット](https://www.cs.toronto.edu/~kriz/cifar.html)は，飛行機や犬などの物体が表示されている画像から構成されたデータセットである．

CIFAR10では，10クラスの画像データから構成されています．

![CIFAR10_sample.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/176458/b6b43478-c85f-9211-7bc6-227d9b387af5.png)

呼び出しには`CIFAR10()`を使用します．
引数については，前述のMNISTと同様ですので，説明は割愛します．

In [ ]:
train_data = torchvision.datasets.CIFAR10(root="./", train=True, transform=transforms.ToTensor(), download=True)
test_data = torchvision.datasets.CIFAR10(root="./", train=False, transform=transforms.ToTensor(), download=True)

print(len(train_data))
print(len(test_data))

# DataLoaderの定義
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=10, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=10, shuffle=False)

for image, label in train_loader:
    print(image.size())
    print(label.size(), label)
    break

---
## CIFAR100データセット

CIFAR100は100クラスの画像データから構成されるデータセットである．

![1fTQtXyApxWPoW2vzSEk_Pw_t41Ubes.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/143078/484c6678-85f6-341e-1e63-d686e3ba9c47.png)

CIFAR100には，各物体のクラスだけではなく，その上位クラスであるsuperclassが用意されている．
superclassの数は20であり，それぞれのsuperclassは5種類のクラスから構成されている．

| Superclass | Classes |
|-------------------|----------|
| aquatic mammals | beaver, dolphin, otter, seal, whale |
| fish | aquarium fish, flatfish, ray, shark, trout |
| flowers | orchids, poppies, roses, sunflowers, tulips |
| food containers | bottles, bowls, cans, cups, plates |
| fruit and vegetables | apples, mushrooms, oranges, pears, sweet peppers |
| household electrical devices | clock, computer keyboard, lamp, telephone, television |
| household furniture | bed, chair, couch, table, wardrobe |
| insects | bee, beetle, butterfly, caterpillar, cockroach |
| large carnivores | bear, leopard, lion, tiger, wolf |
| large man-made outdoor things | bridge, castle, house, road, skyscraper |
| large natural outdoor scenes | cloud, forest, mountain, plain, sea |
| large omnivores and herbivores | camel, cattle, chimpanzee, elephant, kangaroo |
| medium-sized mammals | fox, porcupine, possum, raccoon, skunk |
| non-insect invertebrates | crab, lobster, snail, spider, worm |
| people | baby, boy, girl, man, woman |
| reptiles | crocodile, dinosaur, lizard, snake, turtle |
| small mammals | hamster, mouse, rabbit, shrew, squirrel |
| trees | maple, oak, palm, pine, willow |
| vehicles 1 | bicycle, bus, motorcycle, pickup truck, train |
| vehicles 2 | lawn-mower, rocket, streetcar, tank, tractor |

呼び出しには`CIFAR100()`を使用します．
引数については，前述のMNISTと同様ですので，説明は割愛します．

※ superclassの情報は`torchvision.datasets.CIFAR100`には含まれていません．
必要な場合は，こちらの[GitHub](https://github.com/ryanchankh/cifar100coarse)などを参考にご活用ください．

In [ ]:
train_data = torchvision.datasets.CIFAR100(root="./", train=True, transform=transforms.ToTensor(), download=True)
test_data = torchvision.datasets.CIFAR100(root="./", train=False, transform=transforms.ToTensor(), download=True)

print(len(train_data))
print(len(test_data))

# DataLoaderの定義
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=10, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=10, shuffle=False)

for image, label in train_loader:
    print(image.size())
    print(label.size(), label)
    break

---
## ImageNet & ILSVRC

### ImageNet
[ImageNet](https://image-net.org/index.php)は，Stanford大学のFei-Fei Liらのを中心としたグループが作成・管理している画像データベースです．
ImageNetは1400万枚を超える画像から構成されています．
各画像には物体のクラスが付与されており，ImageNet内のクラス数は2万種類以上となっています．

これらの物体クラスは，[WordNet](https://wordnet.princeton.edu/)と呼ばれる英語の語彙のデータベースを基に作成されています．
WordNetは**Synset**と呼ばれる同義語 (synonyms) セットがツリー階層構造でグループを形成しながら定義されている．
ImageNetではその2万以上のsynsetが採用されています．

![ImageNet](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/143078/223f3a75-75ed-8601-a017-02cf84d5d31e.jpeg)


### ImageNet Large Scale Visual Recognition Challenge (ILSVRC)

[ImageNet Large Scale Visual Recognition Challenge (ILSVRC)](https://www.image-net.org/challenges/LSVRC/) は，2010年から2017年まで開催された大規模画像認識コンペです．
ILSVRCでは，上で紹介したImageNetの1400万枚の画像全てを使用するわけではなく，一部の画像データを使用して学習や評価を行なっています．

また，ILSVRCにはいくつかのタスクが設けられています．

1. Classification (2010 ~)
2. Classification with Localizaiton (2011 ~)
3. Fine-grained Classification (2012 ~)
4. Detection (2013 ~)
5. Object Detection from Video (2015 ~)
6. Scene classification (2015, 2016)
7. Scene Parsing (2016)

このうち，1.のClassificationタスクが最も有名なタスクです．
Classificationタスクは，1000カテゴリの物体クラスを分類するタスクであり，学習用データ120万枚，検証用データ5万枚，テストデータ10万枚のデータから構成されています．

2010年からスタートしたコンペタスクですが．2012年に開催されたILSVRC 2012にて，CNNモデル (AlexNet) が高い認識精度（低いエラー率）で優勝したことをキッカケに，ディープラーニングが注目を浴びました．
それ以降，様々なネットワークが提案され，このILSVRC2012で使用されたデータセットを用いて認識性能向上を図っています．
このILSVRC2012で用いられたデータセットを通称 **ImageNet** と呼び，広く用いられています．

最後のILSVRCとなった2017年以降は，画像認識コンペの役割を[Kaggle](https://www.kaggle.com/c/imagenet-object-localization-challenge/overview/description)に譲渡しています．



### ImageNetのPyTorchでの使用方法

`torchvision.datasets`にImageNetのデータセットクラスは用意されていますが，

* ImageNetは非常に大規模なデータセットであること
* ダウンロードにはユーザー登録が必要なこと

から前述のMNISTやCIFARのように自動でダウンロードを行なってはくれません．
データの準備に非常に時間がかかるため，ここではImageNetを使用するための手順について説明し，最後にデータセットを呼び出すためのスクリプトを紹介するのみとします．

#### 1. ImageNet公式サイトへのユーザー登録

まず，[ImageNetの公式サイト](https://image-net.org/index.php)へ移動し，ユーザー登録を行います．
画面右上あたりに`signup`のリンクがありますので，そこからユーザー登録を行ってください．

※ すでにユーザー登録を行っている人は，ログインを行ってください．

#### 2. ImageNetデータセットのダウンロードと展開

次に．[ILSVRC2012のデータセットダウンロードページ](https://image-net.org/challenges/LSVRC/2012/2012-downloads.php)へ移動し，
* Development kit (Task 1 & 2). 2.5MB.
* Training images (Task 1 & 2). 138GB. MD5: 1d675b47d978889d74fa0da5fadfb00e
* Validation images (all tasks). 6.3GB. MD5: 29b22e2961454d5413ddabcf34fc5622

の3つをダウンロードします．

* Test images (all tasks). 13GB. MD5: e1b8681fff3d63731c599df9b4b6fc02

はtorchvisionでは使用できませんが，必要な場合はダウンロードしてください．

**※ ダウンロードには1日以上かかると思われますので，ご注意ください．**

ダウンロードが完了したら，`*.tar.gz`ファイルを展開します．

展開した学習データのファイル（フォルダ）の中身を確認すると，`n01440764`や`n01443537`などの名前のフォルダが存在しており，そのフォルダの中に様々な画像データが格納されていることがわかります．
学習用画像では，画像データがクラス毎にフォルダ分けされています．
このフォルダの番号は，WordNet IDと呼ばれるもので，クラス識別のためのIDとなっています．

#### 3. ImageNetクラスの呼び出し

`torchvision.datasets.ImageNet`を用いて，ImageNetデータセットを使用できるよう呼び出します．

`root`は先ほどダウンロード，展開したデータセットがあるディレクトリまでのパスを指定します．
`split`は学習用 (`"train"`) または検証用 (`"val"`) を指定します．
`transform`はデータを呼び出す際に行う前処理やaugmentation指定する部分です．
`download`はデータセットをダウンロードするかどうかの引数ですが，`True`を指定した場合でもエラーメッセージを出力して終了しますので使用しないよう注意してください．



In [ ]:
imagenet_train_data = torchvision.datasets.ImageNet(root='path/to/imagenet_root/', split="train", transform=transforms.ToTensor())
imagenet_val_data = torchvision.datasets.ImageNet(root='path/to/imagenet_root/', split="val", transform=transforms.ToTensor())

print(len(imagenet_train_data))
print(len(imagenet_val_data))

# DataLoaderの定義
imagenet_train_loader = torch.utils.data.DataLoader(dataset=imagenet_train_data, batch_size=256, shuffle=True)
imagenet_val_loader = torch.utils.data.DataLoader(dataset=imagenet_test_data, batch_size=256, shuffle=False)

for image, label in imagenet_train_loader:
    print(image.size())
    print(label.size(), label)
    break